In [ ]:
# Build AdaBoost Model from scratch

In [ ]:
from numpy import *

## Step 1. Build AdaBoost Model

### 1.1 Load Data

In [ ]:
# generate a simple dataset
def loadSimpData():
    """ Test Data
    Returns:
        dataArr  
        labelArr
    """
    dataArr = array([[1., 2.1], [2., 1.1], [1.3, 1.], [1., 1.], [2., 1.]])
    labelArr = [1.0, 1.0, -1.0, -1.0, 1.0]
    return dataArr, labelArr

In [ ]:
# general function to parse tab -delimited floats
def loadDataSet(fileName):
    # get number of fields
    numFeat = len(open(fileName).readline().split('\t'))
    dataArr = []
    labelArr = []
    fr = open(fileName)
    for line in fr.readlines():
        lineArr = []
        curLine = line.strip().split('\t')
        for i in range(numFeat-1):
            lineArr.append(float(curLine[i]))
        dataArr.append(lineArr)
        labelArr.append(float(curLine[-1]))
    return dataArr, labelArr

### 1.2 Build Weak Classifier (Decision stump-generating functions)

In [ ]:
def stumpClassify(dataMat, dimen, threshVal, threshIneq):
    """stumpClassify
    Args:
        dataMat    Matrix
        dimen     Feature Column
        threshVal   The value needed to be compared
    Returns:
        retArray 
    """
    # Default num is 1
    retArray = ones((shape(dataMat)[0], 1))
    # dataMat[:, dimen] shows all values in Column [dimen]

    if threshIneq == 'lt':
        retArray[dataMat[:, dimen] <= threshVal] = -1.0
    else:
        retArray[dataMat[:, dimen] > threshVal] = -1.0
    return retArray

In [ ]:
def buildStump(dataArr, labelArr, D):
    """buildStump(Decision Stump)
    Args:
        dataArr   
        labelArr  
        D         The weight vector
    Returns:
        bestStump    Best Decision Stump model
        minError    Minimum Error rate
        bestClasEst  The result set after training
    """
    # Data Transform
    dataMat = mat(dataArr)
    labelMat = mat(labelArr).T
    # Row m, Column n
    m, n = shape(dataMat)

    # Initialize Data
    numSteps = 10.0
    bestStump = {}
    bestClasEst = mat(zeros((m, 1)))
    # Initialized minError is infinite
    minError = inf

    # Circulate all feature columns
    for i in range(n):
        rangeMin = dataMat[:, i].min()
        rangeMax = dataMat[:, i].max()
        # print 'rangeMin=%s, rangeMax=%s' % (rangeMin, rangeMax)
        stepSize = (rangeMax-rangeMin)/numSteps
        for j in range(-1, int(numSteps)+1):
            # go over less than and greater than
            for inequal in ['lt', 'gt']:
                # if -1, then get rangeMin-stepSize; if numSteps, then get rangeMax
                threshVal = (rangeMin + float(j) * stepSize)
                # Use decision stumpe model to get classification
                predictedVals = stumpClassify(dataMat, i, threshVal, inequal)
                # print predictedVals
                errArr = mat(ones((m, 1)))
                # Correct is 0, while wrong is 1
                errArr[predictedVals == labelMat] = 0
                weightedError = D.T*errArr
                '''
                dim         represent feature columns
                threshVal      threshold value of tree
                inequal       represent the situation when not equal
                weightedError  show error rate of all results
                bestClasEst   best predicted result
                '''
                # print "split: dim %d, thresh %.2f, thresh ineqal: %s, the weighted error is %.3f" % (i, threshVal, inequal, weightedError)
                if weightedError < minError:
                    minError = weightedError
                    bestClasEst = predictedVals.copy()
                    bestStump['dim'] = i
                    bestStump['thresh'] = threshVal
                    bestStump['ineq'] = inequal
    return bestStump, minError, bestClasEst

### 1.3 AdaBoost Training with Decision stumps

In [ ]:
def adaBoostTrainDS(dataArr, labelArr, numIt=40):
    """adaBoostTrainDS
    Args:
        dataArr   
        labelArr  
        numIt    
    Returns:
        weakClassArr  set of weak classifiers
        aggClassEst   predicted results of classification
    """
    weakClassArr = []
    m = shape(dataArr)[0]
    D = mat(ones((m, 1))/m)
    aggClassEst = mat(zeros((m, 1)))
    for i in range(numIt):
        # get the decision stump model
        bestStump, error, classEst = buildStump(dataArr, labelArr, D)

        # The alpha values are based on the error of each weak classifier
        # calculate alpha value
        alpha = float(0.5*log((1.0-error)/max(error, 1e-16)))
        bestStump['alpha'] = alpha
        # store Stump Params in Array
        weakClassArr.append(bestStump)

        # print "alpha=%s, classEst=%s, bestStump=%s, error=%s " % (alpha, classEst.T, bestStump, error)
        expon = multiply(-1*alpha*mat(labelArr).T, classEst)
        # print '\n'
        # print 'labelArr=', labelArr
        # print 'classEst=', classEst.T
        # print '\n'
        # print 'multi: ', multiply(mat(labelArr).T, classEst).T
        D = multiply(D, exp(expon))
        D = D/D.sum()
        # print "D: ", D.T
        # print '\n'

        aggClassEst += alpha*classEst
        # use sign() function to make positive number as 1, negative number as -1, and 0 as 0
        aggErrors = multiply(sign(aggClassEst) != mat(labelArr).T, ones((m, 1)))
        errorRate = aggErrors.sum()/m
        # print "total error=%s " % (errorRate)
        if errorRate == 0.0:
            break
    return weakClassArr, aggClassEst

### 1.4 AdaBoost Classification Function

In [ ]:
def adaClassify(datToClass, classifierArr):
    # do stuff similar to last aggClassEst in adaBoostTrainDS
    dataMat = mat(datToClass)
    m = shape(dataMat)[0]
    aggClassEst = mat(zeros((m, 1)))

    # Circulate different classifiers
    for i in range(len(classifierArr)):
        classEst = stumpClassify(dataMat, classifierArr[i]['dim'], classifierArr[i]['thresh'], classifierArr[i]['ineq'])
        aggClassEst += classifierArr[i]['alpha']*classEst
        # print aggClassEst
    return sign(aggClassEst)

### 1.5 ROC plotting and AUC calculating function

In [ ]:
def plotROC(predStrengths, classLabels):
    """plotROC(Print the ROC curve and calculate the AUC (area under the curve))
    Args:
        predStrengths  Weighted values of predicted results
        classLabels   Original set of labels
    """
    print('predStrengths=', predStrengths)
    print('classLabels=', classLabels)

    import matplotlib.pyplot as plt
    # variable to calculate AUC
    ySum = 0.0
    # Sum positive samples
    numPosClas = sum(array(classLabels)==1.0)
    # Percentage of positive samples
    yStep = 1/float(numPosClas)
    # Percentage of negative samples
    xStep = 1/float(len(classLabels)-numPosClas)
    # Use argsort() function to get sorted index, it's reverse
    sortedIndicies = predStrengths.argsort()
    # Test the sequence of the results
    print('sortedIndicies=', sortedIndicies, predStrengths[0, 176], predStrengths.min(), predStrengths[0, 293], predStrengths.max())

    # create figures for the plot
    fig = plt.figure()
    fig.clf()
    ax = plt.subplot(111)
    # cursor
    cur = (1.0, 1.0)
    # loop through all the values, drawing a line segment at each point
    for index in sortedIndicies.tolist()[0]:
        if classLabels[index] == 1.0:
            delX = 0
            delY = yStep
        else:
            delX = xStep
            delY = 0
            ySum += cur[1]
        # draw line from cur to (cur[0]-delX, cur[1]-delY)
        print(cur[0], cur[0]-delX, cur[1], cur[1]-delY)
        ax.plot([cur[0], cur[0]-delX], [cur[1], cur[1]-delY], c='b')
        cur = (cur[0]-delX, cur[1]-delY)
    # draw diagonal dotted line 
    ax.plot([0, 1], [0, 1], 'b--')
    plt.xlabel('False positive rate')
    plt.ylabel('True positive rate')
    plt.title('ROC curve for AdaBoost horse colic detection system')
    # set the drawing area
    ax.axis([0, 1, 0, 1])
    plt.show()
    print("the Area Under the Curve is: ", ySum*xStep)

## Step 2. AdaBoost on a simple dataset

In [ ]:
# Simple Dataset

# Classify 5 dots
dataArr, labelArr = loadSimpData()
print('dataArr:\n', dataArr, '\nlabelArr:', labelArr)

# Initialized D value
# weightedError = D.T*errArr
D = mat(ones((5, 1))/5)
print('\nD=', D.T)

bestStump, minError, bestClasEst = buildStump(dataArr, labelArr, D)
print('\nbestStump =', bestStump)
print('\nminError =', minError)
print('\nbestClasEst =', bestClasEst.T)

# weakClassArr
weakClassArr, aggClassEst = adaBoostTrainDS(dataArr, labelArr, 9)
print('\nweakClassArr =', weakClassArr, '\naggClassEst =', aggClassEst.T)

# Results
print(adaClassify([0, 0], weakClassArr).T)
print(adaClassify([[5, 5], [0, 0]], weakClassArr).T)

## Step 3. AdaBoost on a difficult dataset (Horse Colic Dataset)

### 3.1 Train Classifiers 

In [ ]:
# Difficult Dataset (Horse Colic Dataset)
# Training Set
dataArr, labelArr = loadDataSet("drive/MyDrive/horseColicTraining2.txt")
weakClassArr, aggClassEst = adaBoostTrainDS(dataArr, labelArr, 40)
print(weakClassArr, 
      '\n-------------------------------------------------------------------\n', aggClassEst.T, 
      '\n-------------------------------------------------------------------\n')

# Get ROC and AUC
plotROC(aggClassEst.T, labelArr)

### 3.2 Test the AdaBoost Model

In [ ]:
# Test Set
dataArrTest, labelArrTest = loadDataSet("drive/MyDrive/horseColicTest2.txt")
m = shape(dataArrTest)[0]
predicting10 = adaClassify(dataArrTest, weakClassArr)
errArr = mat(ones((m, 1)))
# Test result: The number of samples, The number of wrong predictions, Error Rate
print('The Number of Test Samples:', m, 
    '\nThe Number of Wrong Predictions:', errArr[predicting10 != mat(labelArrTest).T].sum(), 
    '\nError Rate:', errArr[predicting10 != mat(labelArrTest).T].sum()/m)